<a href="https://www.kaggle.com/code/dplayer/task-2-rentalpricedata-exploration?scriptVersionId=100789050" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/rental-ca-vacant-market-rentalprices-data/May2022-Table 1.csv
/kaggle/input/rental-ca-vacant-market-rentalprices-data/June2022-June2022.csv
/kaggle/input/rental-ca-vacant-market-rentalprices-data/Oct2021-Table 1.csv
/kaggle/input/rental-ca-vacant-market-rentalprices-data/Aug2021-Table 1.csv
/kaggle/input/rental-ca-vacant-market-rentalprices-data/Nov2021-Table 1.csv
/kaggle/input/rental-ca-vacant-market-rentalprices-data/Feb2022-Feb2022.csv
/kaggle/input/rental-ca-vacant-market-rentalprices-data/Apr2022-Apr2022.csv
/kaggle/input/rental-ca-vacant-market-rentalprices-data/Sep2021-Table 1.csv
/kaggle/input/rental-ca-vacant-market-rentalprices-data/Dec2021-Table 1.csv
/kaggle/input/rental-ca-vacant-market-rentalprices-data/Mar2022-Mar2022.csv
/kaggle/input/rental-ca-vacant-market-rentalprices-data/Jan2022-Jan2022.csv
/kaggle/input/housing-affordability-in-canada/Structural-dwellings-household-size.csv
/kaggle/input/housing-affordability-in-canada/HPI 1981-2022 federal and prov

Load data

## Context
rental.ca generates a rental report every month and summarizes rental prices for vacant properties on the market for rent. They list the average price, average month over month (MOM)% and average year over year (YOY)% for 1 bedroom and 2 bedroom properties for major cities. The following data is colleceted from August 2021 until June 2022.

In this notebook I have combined all the data into asingle table.

In [3]:
## imports
from pathlib import Path

In [4]:
import re
p = re.compile(r'\D')

Added a simple cleanup function to remove unnecessary characters within the data. I also add  2 more columns for month and year.

In [5]:
def cleanupandadddate(file_name, month, year):
    df = pd.read_csv(Path(file_name))
    #print(df)
    try:
        df['1 BED'] = [p.sub('', x) for x in df['1 BED']]
    except TypeError as te:
        print(f"1-BED {te}")
    try:
        df['2 BED'] = [p.sub('', x) for x in df['2 BED']]
    except TypeError as te:
        print(f"2-BED {te}")
    try:
        df['M/M'] = [re.sub(r'%', '', str(x)) for x in df['M/M']]
    except TypeError as te:
        print(f"M/M {te}")
    try:
        df['Y/Y'] = [re.sub(r'%', '', str(x)) for x in df['Y/Y']]
    except TypeError as te:
        print(f"Y/Y {te}")
    try:
        df['M/M.1'] = [re.sub(r'%', '', str(x)) for x in df['M/M.1']]
    except TypeError as te:
        print(f"M/M.1 {te}")
    try:
        df['Y/Y.1'] = [re.sub(r'%', '', str(x)) for x in df['Y/Y.1']]
    except TypeError as te:
        print(f"Y/Y.1 {te}")
    df['Month'] =[month]*df.shape[0]
    df['Year'] = [year]*df.shape[0]
    df.columns = ['City', '1-Bed($)', 'M/M-1-BED(%)', 'Y/Y-1-BED(%)', '2-BED', 'M/M-2-BED(%)', 'Y/Y-2-BED(%)', 'Month', 'Year']
    return df

In [6]:
updfMay = cleanupandadddate('/kaggle/input/rental-ca-vacant-market-rentalprices-data/May2022-Table 1.csv', 'May', '2022')
print(updfMay)

              City 1-Bed($) M/M-1-BED(%) Y/Y-1-BED(%) 2-BED M/M-2-BED(%)  \
0        Vancouver     2334         2.37        18.12  3324         6.47   
1          Toronto     2065         2.08        12.96  2849         2.63   
2       Burlington     2017         0.00        15.85  2261         0.22   
3         Oakville     2010         5.29         8.12  2333         0.09   
4        Etobicoke     1897         2.49        10.68  2446         4.80   
5          Burnaby     1883        -3.88        15.24  2563         5.39   
6      Mississauga     1817         0.11         5.27  2211         0.32   
7           Guelph     1811         3.37        12.91  2179        -0.23   
8      Victoria BC     1776         0.74         6.86  2736         7.67   
9           Barrie     1771         0.11         2.02  2084         0.82   
10         Vaughan     1730         6.86        11.54  2338         6.08   
11      North York     1725         1.29          4.1  2259         3.62   
12       Kit

In [7]:
updfApr = cleanupandadddate('/kaggle/input/rental-ca-vacant-market-rentalprices-data/Apr2022-Apr2022.csv', 'April', '2022')
print(updfApr)

              City 1-Bed($) M/M-1-BED(%) Y/Y-1-BED(%) 2-BED M/M-2-BED(%)  \
0        Vancouver     2280         1.83        13.94  3122         2.36   
1          Toronto     2023        -1.03        10.97  2776        -0.07   
2       Burlington     2017         8.67        12.74  2256        -1.01   
3          Burnaby     1959        -1.01        16.47  2432         1.42   
4         Oakville     1909        -0.68         8.71  2331        -3.72   
5        Etobicoke     1851        -0.48         8.18  2334        -4.81   
6      Mississauga     1815         2.20         4.01  2204         0.50   
7           Barrie     1769        -0.11         3.21  2067         1.87   
8         Victoria     1763         6.14         7.63  2541        -4.72   
9           Guelph     1752         1.33          9.3  2184         3.41   
10          Oshawa     1657        -9.06         18.7  1906         3.25   
11            Aiax     1650         2.61         2.42  1908        -1.29   
12         H

In [8]:
df = pd.read_csv(Path('/kaggle/input/rental-ca-vacant-market-rentalprices-data/Aug2021-Table 1.csv'))
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35 entries, 0 to 34
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   City    35 non-null     object
 1   1 BED   35 non-null     object
 2   M/M     35 non-null     object
 3   Y/Y     34 non-null     object
 4   2 BED   35 non-null     int64 
 5   M/M.1   35 non-null     object
 6   Y/Y.1   34 non-null     object
dtypes: int64(1), object(6)
memory usage: 2.0+ KB


In [9]:
updfAug = cleanupandadddate('/kaggle/input/rental-ca-vacant-market-rentalprices-data/Aug2021-Table 1.csv', 'August', '2021')
print(updfAug)

2-BED expected string or bytes-like object
               City 1-Bed($) M/M-1-BED(%) Y/Y-1-BED(%)  2-BED M/M-2-BED(%)  \
0     Vancouver. BC     2185          6.3         14.4   3041           10   
1           Toronto     1855          1.0         -9.0   2606          4.2   
2         Etobicoke     1802          2.8         -7.2   2361          3.6   
3          Victoria     1756          8.7         11.6   2103          2.0   
4       Mississauga     1753         -0.3         -9.3   2129          1.6   
5        Burlington     1745         -2.2        -10.7   2065         -0.8   
6            Guelph     1714          2.2         10.2   2000          2.1   
7           Vaughan     1699          9.2         -9.2   2115         -2.3   
8           Burnaby     1672          0.2         -3.7   2244          0.1   
9        North York     1669         -0.1         -9.7   2025         -0.3   
10  New Westminster     1668          4.8         10.9   2010         -5.7   
11           Ottawa  

In [10]:
updfDec = cleanupandadddate('/kaggle/input/rental-ca-vacant-market-rentalprices-data/Dec2021-Table 1.csv', 'December', '2021')
print(updfDec)

1-BED expected string or bytes-like object
               City  1-Bed($) M/M-1-BED(%) Y/Y-1-BED(%) 2-BED M/M-2-BED(%)  \
0         Vancouver      2132        -3.79         7.73  2998        -1.93   
1           Toronto      2040         1.69         7.99  2764         3.25   
2          Oakville      1879        -5.29        -1.78  2322        -1.02   
3        Burlington      1827        -3.23         4.09  2275         1.38   
4         Etobicoke      1809         2.26        -0.39  2302        -0.04   
5       Mississauga      1796        -1.54        -2.55  2142        -0.83   
6            Barrie      1746       -12.87        14.27  1989        -8.93   
7           Markham      1715        -0.52         1.96  1972        -1.55   
8            Guelph      1704        -3.51        12.25  2118         0.14   
9            Oshawa      1701          nan        13.25  1783        -3.83   
10          Burnaby      1696        -3.58        -0.47  2414         5.88   
11         Victoria  

In [11]:
updfFeb = cleanupandadddate('/kaggle/input/rental-ca-vacant-market-rentalprices-data/Feb2022-Feb2022.csv', 'February', '2022')
print(updfFeb)

1-BED expected string or bytes-like object
2-BED expected string or bytes-like object
              City  1-Bed($) M/M-1-BED(%) Y/Y-1-BED(%)  2-BED M/M-2-BED(%)  \
0        Vancouver      2163        -0.60         9.24   3003         0.67   
1          Toronto      2027         0.70        11.50   2769         1.99   
2         Oakville      1959         2.83         9.69   2388        -0.50   
3        Etobicoke      1857         0.00         5.09   2424         0.66   
4       Burlington      1851         0.71         0.00   2317        -0.17   
5           Oshawa      1849        -3.55        20.93   1739        -1.86   
6          Burnabv      1847        -7.23         7.38   2377        -0.79   
7    Richmond Hill      1806         0.61         7.31   2254         0.67   
8      Mississauga      1778           34         0.23   2179         1.82   
9   North York. ON      1705         1.55         1.07   2127        -0.42   
10          Ottawa      1689         4.39         6.76  

In [12]:
updfJan = cleanupandadddate('/kaggle/input/rental-ca-vacant-market-rentalprices-data/Jan2022-Jan2022.csv', 'January', '2022')
print(updfJan)

1-BED expected string or bytes-like object
               City  1-Bed($) M/M-1-BED(%) Y/Y-1-BED(%) 2-BED M/M-2-BED(%)  \
0         Vancouver      2176         2.06        13.22  2983        -0.50   
1           Toronto      2013        -1.42         9.22  2715        -1.95   
2         Etobicoke      1857         2.94         4.09  2408         5.24   
3       Mississauga      1772        -1.34        -2.32  2140        -0.09   
4          York. ON      1706         1.73        -1.78  2250         4.17   
5        North York      1679         1.27        -2.21  2136         0.75   
6         Brantford      1648         3.13        11.73  1767        -0.62   
7          Brampton      1621         4.45         7.14  1884         0.48   
8            Ottawa      1618         1.13         2.21  2023        -0.20   
9              Ajax      1617         5.89        -0.92  1956         0.15   
10          Vaughan      1602        -3.78        -4.98  2159         4.86   
11          Halifax  

In [13]:
updfJun = cleanupandadddate('/kaggle/input/rental-ca-vacant-market-rentalprices-data/June2022-June2022.csv', 'June', '2022')
print(updfJun)

              City 1-Bed($) M/M-1-BED(%) Y/Y-1-BED(%) 2-BED M/M-2-BED(%)  \
0        Vancouver     2377          1.8         19.9  3495          5.1   
1          Toronto     2133          3.3         15.7  3002          5.4   
2          Burnaby     2012          6.9         24.1  2645          3.2   
3         Oakville     2011          0.0          8.2  2445          4.8   
4       Burlington     1978         -1.9         11.0  2331          3.1   
5        Etobicoke     1972          4.0         14.0  2586          5.7   
6           Guelph     1950          7.7         19.6  2156          -11   
7    Richmond Hill     1928          2.9         20.3  2296          5.3   
8         Victoria     1870          5.3         13.4  2896          5.8   
9      Mississauga     1855          2.1          5.6  2282          3.2   
10       Kitchener     1839          8.1         22.4  2179          7.0   
11      North York     1781          3.2           79  2367          4.8   
12         V

In [14]:
updfMar = cleanupandadddate('/kaggle/input/rental-ca-vacant-market-rentalprices-data/Mar2022-Mar2022.csv', 'March', '2022')
print(updfMar)

              City 1-Bed($) M/M-1-BED(%) Y/Y-1-BED(%) 2-BED M/M-2-BED(%)  \
0        Vancouver     2239         3.51        14.70  3050         1.57   
1          Toronto     2044         0.39        12.68  2778        -0.71   
2          Burnaby     1979         7.15        16.82  2398         0.88   
3         Oakville     1922        -1.89         9.77  2421         1.38   
4        Etobicoke     1860         0.00         6.41  2452         0.91   
5           Oshawa     1822        -1.46          nan  1846         6.15   
6      Mississauga     1776        -0.11         0.91  2193         0.64   
7           Barrie     1771          nan          nan  2029         0.84   
8           Guelph     1729         6.20         9.50  2112         2.33   
9   North York. ON     1724         1.11         1.89  2125        -0.09   
10          Ottawa     1692         0.18         9.87  2033        -1.12   
11         Vaughan     1671         4.24         2.20  2208         3.08   
12         H

In [15]:
updfNov = cleanupandadddate('/kaggle/input/rental-ca-vacant-market-rentalprices-data/Nov2021-Table 1.csv', 'November', '2021')
print(updfNov)

2-BED expected string or bytes-like object
               City 1-Bed($) M/M-1-BED(%) Y/Y-1-BED(%)  2-BED M/M-2-BED(%)  \
0         Vancouver     2216          2.8         16.6   3057          4.3   
1           Toronto     2006          3.5          4.3   2678          1.9   
2          Oakville     1984          0.3          2.9   2346          0.0   
3        Burlington     1888          3.5          2.1   2244          1.6   
4      Victoria. BC     1863          8.6         19.4   2432         -7.0   
5           Vaughan     1828         -1.6         11.6   2077         -2.1   
6       Mississauga     1824          0.9         -2.8   2160         -0.9   
7         Etobicoke     1769         -1.6         -5.0   2303         -2.4   
8            Guelph     1766          3.5         11.3   2115          5.3   
9           Burnaby     1759         -4.0          6.7   2280         -3.6   
10      Markham. ON     1724         -4.4         -2.6   2003         -7.2   
11       North York  

In [16]:
updfOct = cleanupandadddate('/kaggle/input/rental-ca-vacant-market-rentalprices-data/Oct2021-Table 1.csv', 'October', '2021')
print(updfOct)

                 City 1-Bed($) M/M-1-BED(%) Y/Y-1-BED(%) 2-BED M/M-2-BED(%)  \
0           Vancouver     2155         -0.5         11.0  2932         -3.7   
1             Toronto     1938         -2.6         -1.5  2628          0.0   
2             Vaughan     1858          0.4          7.4  2122         -0.1   
3             Burnaby     1833          3.8          8.5  2366          1.0   
4          Burlington     1823         -0.5         -3.2  2209          0.2   
5         Mississauga     1809          1.1         -6.4  2180         -0.3   
6             Markham     1803          2.5          2.3  2157          9.3   
7           Etobicoke     1797         -1.1         -4.9  2360          4.0   
8            Victoria     1715          nan          6.1  2614          nan   
9      North York. ON     1714          1.9         -5.7  2086          1.8   
10             Guelph     1706         -1.3         10.7  2009         -1.3   
11         Gloucester     1689          1.7         

In [17]:
updfSep = cleanupandadddate('/kaggle/input/rental-ca-vacant-market-rentalprices-data/Sep2021-Table 1.csv', 'September', '2021')
print(updfSep)

1-BED expected string or bytes-like object
2-BED expected string or bytes-like object
               City  1-Bed($) M/M-1-BED(%) Y/Y-1-BED(%)  2-BED M/M-2-BED(%)  \
0         Vancouver      2167         -0.8         15.7   3044          0.1   
1           Toronto      1989          7.2         -1.2   2628          0.8   
2           Vaughan      1850          8.9          5.3   2124          0.5   
3        Burlington      1832          5.0         -4.9   2205          6.8   
4         Etobicoke      1816          0.8         -6.2   2269         -3.9   
5       Mississauga      1790          2.1         -7.2   2187          2.7   
6           Burnaby      1765          5.6          5.1   2341          4.3   
7           Markham      1760         -1.4         -4.7   1974         -0.1   
8            Guelph      1728          0.8         13.4   2035          1.8   
9        North York      1681          0.8        -10.0   2049          1.2   
10  New Westminster      1623         -2.7   

## Combine the data here

In [18]:
full_data = pd.concat([updfAug, updfSep, updfOct, updfNov, updfDec, updfJan, updfFeb, updfMar, updfApr, updfMay, updfJun])

In [19]:
print(full_data)

             City 1-Bed($) M/M-1-BED(%) Y/Y-1-BED(%) 2-BED M/M-2-BED(%)  \
0   Vancouver. BC     2185          6.3         14.4  3041           10   
1         Toronto     1855          1.0         -9.0  2606          4.2   
2       Etobicoke     1802          2.8         -7.2  2361          3.6   
3        Victoria     1756          8.7         11.6  2103          2.0   
4     Mississauga     1753         -0.3         -9.3  2129          1.6   
..            ...      ...          ...          ...   ...          ...   
30       Edmonton     1043         -0.7          4.7  1309            1   
31     Lethbridge     1043         -3.3          9.0  1210          0.9   
32         Regina      990          4.7          1.7  1209          3.4   
33       Red Deer      988          3.7          121  1147          5.5   
34      Saskatoon      975         -1.1          5.0  1145          0.6   

   Y/Y-2-BED(%)   Month  Year  
0          14.5  August  2021  
1          -3.2  August  2021  
2  

In [20]:
edited_full_data = full_data

In [21]:
edited_full_data.loc[edited_full_data['City'].str.contains("Surrev", case=False), 'City'] = 'Surrey'

In [22]:
edited_full_data.loc[edited_full_data['City'].str.contains("Victoria", case=False), 'City'] = 'Victoria'

In [23]:
edited_full_data.loc[edited_full_data['City'].str.contains("VAncouver", case=False), 'City'] = 'Vancouver'

In [24]:
edited_full_data.loc[edited_full_data['City'].str.contains("Oshawa", case=False), 'City'] = 'Oshawa'

In [25]:
edited_full_data.loc[edited_full_data['City'].str.contains("Saint", case=False), 'City'] = 'Saint-Laurent' 

In [26]:
edited_full_data.loc[edited_full_data['City'].str.contains("North York", case=False), 'City'] = 'North York'

In [27]:
edited_full_data.loc[edited_full_data['City'].str.contains("Windsor", case=False), 'City'] = 'Windsor'

In [28]:
edited_full_data.loc[edited_full_data['City'].str.contains("Laval", case=False), 'City'] = 'Laval'

In [29]:
edited_full_data.loc[edited_full_data['City'].str.contains("Aiax", case=False), 'City'] = 'Ajax'

In [30]:
edited_full_data.loc[edited_full_data['City'].str.contains("Halifax", case=False), 'City'] = 'Halifax'

In [31]:
edited_full_data.loc[edited_full_data['City'].str.contains("Red", case=False), 'City'] = 'Red Deer'

In [32]:
edited_full_data.loc[edited_full_data['City'].str.contains("Reqina", case=False), 'City'] = 'Regina'

In [33]:
edited_full_data.loc[edited_full_data['City'].str.contains("St.John", case=False), 'City'] = "St. John's"

In [34]:
edited_full_data.loc[edited_full_data['City'].str.contains("Markham", case=False), 'City'] = 'Markham'

In [35]:
edited_full_data.loc[edited_full_data['City'].str.contains("Montreal", case=False), 'City'] = 'Montréal'

In [36]:
edited_full_data.loc[edited_full_data['City'].str.contains("Burnabv", case=False), 'City'] = 'Burnaby'

In [37]:
edited_full_data.loc[edited_full_data['City'].str.contains("Hamilton", case=False), 'City'] = 'Hamilton'

In [38]:
edited_full_data.loc[edited_full_data['City'].str.contains("Edmonton", case=False), 'City'] = 'Edmonton'

In [39]:
edited_full_data.loc[edited_full_data['City'].str.contains("London", case=False), 'City'] = 'London'

In [40]:
edited_full_data.loc[edited_full_data['City'].str.contains("Calaarv", case=False), 'City'] = 'Calgary'

In [41]:
edited_full_data.loc[edited_full_data['City'].str.contains("Gatineau", case=False), 'City'] = 'Gatineau'

In [42]:
edited_full_data.loc[edited_full_data['City'].str.contains("Lethbridge", case=False), 'City'] = 'Lethbridge'

In [43]:
edited_full_data.loc[edited_full_data['City'].str.contains("Saskatoon", case=False), 'City'] = 'Saskatoon'

In [44]:
edited_full_data.loc[edited_full_data['City'].str.contains("Saskatoon", case=False)]

,City,1-Bed($),M/M-1-BED(%),Y/Y-1-BED(%),2-BED,M/M-2-BED(%),Y/Y-2-BED(%),Month,Year
31,Saskatoon,916,-2.6,1.8,1056,-1.4,-5.1,August,2021
32,Saskatoon,890,-2.9,0.7,1046,-1.0,-4.9,September,2021
34,Saskatoon,900,1.2,0.7,1059,1.3,-3.4,October,2021
32,Saskatoon,942,4.6,2.6,1090,3.0,-0.5,November,2021
30,Saskatoon,935,-0.74,3.66,1098,0.73,3.00,December,2021
32,Saskatoon,951,1.71,6.61,1095,-0.27,1.58,January,2022
30,Saskatoon,949,-0.21,3.83,111,1.55,1.37,February,2022
31,Saskatoon,963,1.48,4.11,1130,1.62,3.86,March,2022
31,Saskatoon,984,2.18,12.2,1122,-0.71,4.47,April,2022
32,Saskatoon,986,0.20,6.48,1138,1.43,3.36,May,2022


In [45]:
edited_full_data.City.value_counts()

Vancouver          11
London             11
Winnipeg           11
Montréal           11
Hamilton           11
Kitchener          11
Edmonton           11
Brampton           11
Halifax            11
Toronto            11
Ottawa             11
Saskatoon          11
Mississauga        11
Etobicoke          11
Calgary            11
Regina             10
North York         10
Burnaby            10
Red Deer           10
Oshawa             10
Gatineau           10
Victoria           10
Lethbridge          9
Burlington          9
Vaughan             9
Guelph              9
Laval               8
Windsor             8
Surrey              8
Lloydminster        8
Kingston            8
Oakville            7
Scarborough         7
Markham             6
St. Catharines      6
Grande Prairie      5
Fort McMurray       5
Barrie              5
St. John's          4
New Westminster     4
Nepean              4
Waterloo            3
Gloucester          3
Saint-Laurent       2
Richmond Hill       2
Brantford 

In [46]:
edited_full_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 385 entries, 0 to 34
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   City          385 non-null    object
 1   1-Bed($)      385 non-null    object
 2   M/M-1-BED(%)  385 non-null    object
 3   Y/Y-1-BED(%)  385 non-null    object
 4   2-BED         385 non-null    object
 5   M/M-2-BED(%)  385 non-null    object
 6   Y/Y-2-BED(%)  385 non-null    object
 7   Month         385 non-null    object
 8   Year          385 non-null    object
dtypes: object(9)
memory usage: 30.1+ KB


In [47]:
edited_full_data.to_csv('rental_ca_prices_Aug2021-Jun2022.csv')

### Addendum   

The following data is sourced from a differnt location https://themeasureofaplan.com/rent-prices-versus-income/

They have done some analysis of rent prices vs income. However, their original datasets had a cleaned up version of rent prices which was taken from statcan.
I pulled the rent prices data here.

Below I am putting everything together. I have chosen to build a dataset for 10 years starting from 2010 until 2020. But we can go higher if we want. Just update the parameters and run the notebook again.

In [49]:
## Load the rent prices data
df_2_bed = pd.read_csv(Path('/kaggle/input/rent-data-statcan-1987-2020/Canada_2_Bed_rent.csv'))
df_3_bed = pd.read_csv(Path('/kaggle/input/rent-data-statcan-1987-2020/Canada_3_Bed_rent.csv'))
df_1_bed = pd.read_csv(Path('/kaggle/input/rent-data-statcan-1987-2020/Canada_1Bed_rent.csv'))
df_bachelor = pd.read_csv(Path('/kaggle/input/rent-data-statcan-1987-2020/Canada_bachelor_rent.csv'))

In [50]:
df_2_bed.head()

,City,Province,1987,1988,1989,1990,1991,1992,1993,1994,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
0,St. John's,Newfoundland and Labrador,538,516,535,544,570,575,562,566,...,779,803,871,897,935,986,963,991,980,997
1,Charlottetown,Prince Edward Island,448,457,472,492,514,519,528,529,...,764,808,808,840,832,876,901,922,936,983
2,Cape Breton,Nova Scotia,401,401,424,427,465,474,474,487,...,736,757,760,790,781,774,795,815,847,826
3,Halifax,Nova Scotia,527,537,551,576,604,602,611,616,...,932,960,984,1013,1056,1072,1115,1162,1209,1264
4,Campbellton,New Brunswick part,F,F,F,413,432,434,443,442,...,573,582,583,594,598,608,614,631,639,633


In [67]:
list(range(-4,-2))

[-4, -3]

In [81]:
l = list(range(df_2_bed.columns.get_loc('1987'),df_2_bed.columns.get_loc('1990')))
l.extend([0,1])
l

[2, 3, 4, 0, 1]

In [82]:
df2 = df_2_bed.iloc[: , l].copy()

In [83]:
df2.head()

,1987,1988,1989,City,Province
0,538,516,535,St. John's,Newfoundland and Labrador
1,448,457,472,Charlottetown,Prince Edward Island
2,401,401,424,Cape Breton,Nova Scotia
3,527,537,551,Halifax,Nova Scotia
4,F,F,F,Campbellton,New Brunswick part


In [ ]:
def applyrangeandmelt(df, st_year, en_year):
    

References:

https://rentals.ca/blog/rental-guides - Data scraped from the rental reports from this location. Only the summary table is reproduced for this notebook. Dates are from August 2021 to Jun 2022.